## Задание 1
```
Обязательная часть
Будем парсить страницу со свежими новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. 
Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>
Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>
```

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

KEYWORDS = ['python', 'парсинг']

def has_any_keyword(text, keywords):
    for keyword in keywords:
        if keyword in text:
            return True
    return False
    
def fetch_full_text(article_url):
    req = requests.get(article_url)
    soup = BeautifulSoup(req.text, 'html.parser')
    return soup.find('div', class_='post__body post__body_full').text

req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

articles_dataset = pd.DataFrame(columns = ['дата', 'заголовок', 'ссылка', 'текст'])

articles = soup.find_all('article', class_='post post_preview')
for article in articles:
    article_date = article.find('span', class_ = 'post__time').text
    article_link_node = article.find('a', class_ = 'post__title_link')
    article_title = article_link_node.text
    article_link = article_link_node.get('href')
    article_text = fetch_full_text(article_link)
    if has_any_keyword(article_text, KEYWORDS):
        articles_dataset.loc[len(articles_dataset)] = [article_date, article_title, article_link, article_text]
    time.sleep(0.3)

articles_dataset 

,дата,заголовок,ссылка,текст
0,сегодня в 02:07,Нападения на полицейских в США: статистический...,https://habr.com/ru/post/534134/,\n\r\nЭта статья — логическое продолжение сери...
1,сегодня в 00:30,Как сократить время сборки образов Docker в Gi...,https://habr.com/ru/company/otus/blog/537780/,\nДелаем контейнерные CI среды по-настоящему п...
2,вчера в 21:54,Нейросеть для раскрутки собачьего аккаунта в И...,https://habr.com/ru/post/537714/,\nМеханикаСкрипт для раскрутки назовём для кра...


## Задание 2
```
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.
```

In [4]:
import json
URL = 'https://www.avast.com/hackcheck'

l = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
EMAIL = ['xxx@x.ru', 'yyy@y.com']
params = {
    "emailAddresses": EMAIL
}
headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast'
}
res = requests.post(l, data = json.dumps(params), headers = headers)
json_data = res.json()

In [6]:
json_data_copy = json_data

In [7]:
leak_dataset = pd.DataFrame(columns = ['почта', 'дата утечки', 'источник утечки', 'описание утечки'])

for site, email_breaches in json_data_copy['data'].items():
    for email, breaches in email_breaches.items():
        for breach in breaches:
            breach_info = json_data_copy['breaches'][str(breach['breachId'])]
            leak_dataset.loc[len(leak_dataset)] = [email, breach_info['publishDate'], breach_info['site'], breach_info['description']]         
leak_dataset            
        

,почта,дата утечки,источник утечки,описание утечки
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
1,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
2,yyy@y.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
3,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
4,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
5,yyy@y.com,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
6,yyy@y.com,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
7,yyy@y.com,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
8,yyy@y.com,2017-03-24T00:00:00Z,youku.com,Youku is a large Chinese video content company...
9,yyy@y.com,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
